In [59]:
import pandas as pd
import warnings

In [60]:
# configuração inicial
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', category=DeprecationWarning)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format

In [61]:
# importa os dados do arquivo GeoDados.xlsx
df_ETL = pd.read_excel('GeoDados.xlsx')
df_ETL.head()

,ISO-3166 alpha2,ISO-3166 alpha3,ISO-3166 numeric,fips,País (pt-BR),País (en-US),Capital,Area em km2,População,Continente
0,AD,AND,20,AN,Andorra,Andorra,Andorra la Vella,468.00,77006,EU
1,AE,ARE,784,AE,Emirados Árabes Unidos,United Arab Emirates,Abu Dhabi,82880.00,9630959,AS
2,AF,AFG,4,AF,Afeganistão,Afghanistan,Kabul,647500.00,37172386,AS
3,AG,ATG,28,AC,Antígua e Barbuda,Antigua and Barbuda,St. John's,443.00,96286,NaN
4,AI,AIA,660,AV,Anguila,Anguilla,The Valley,102.00,13254,NaN


In [62]:
df_ETL.columns

Index(['ISO-3166 alpha2', 'ISO-3166 alpha3', 'ISO-3166 numeric', 'fips',
       'País (pt-BR)', 'País (en-US)', 'Capital', 'Area em km2', 'População',
       'Continente'],
      dtype='object')

In [63]:
# contando o número de linhas
df_ETL["País (pt-BR)"].count()

250

In [64]:
# Selecionando as colunas desejadas
colunas_selecionadas = ['ISO-3166 alpha2', 'ISO-3166 alpha3', 'País (pt-BR)', 'Capital']
novo_df = df_ETL[colunas_selecionadas]

novo_df.head()


,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital
0,AD,AND,Andorra,Andorra la Vella
1,AE,ARE,Emirados Árabes Unidos,Abu Dhabi
2,AF,AFG,Afeganistão,Kabul
3,AG,ATG,Antígua e Barbuda,St. John's
4,AI,AIA,Anguila,The Valley


In [65]:
# verificando dados faltantes
novo_df.isnull().sum()

ISO-3166 alpha2    1
ISO-3166 alpha3    0
País (pt-BR)       0
Capital            6
dtype: int64

In [66]:
# exibindo os dados faltantes
novo_df[novo_df['Capital'].isnull()]

,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital
8,AQ,ATA,Antártica,NaN
29,BQ,BES,"Bonaire, Santo Eustáquio e Saba",NaN
33,BV,BVT,Ilha Bouvet,NaN
95,HM,HMD,Ilhas Heard e McDonald,NaN
219,TK,TKL,Toquelau,NaN
231,UM,UMI,Ilhas Distantes dos EUA,NaN


In [67]:
novo_df[novo_df['ISO-3166 alpha2'].isnull()]

,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital
159,NaN,NAM,Namíbia,Windhoek


In [68]:
# Preenchendo capitais faltantes com o nome do país
novo_df['Capital'] = novo_df.apply(
    lambda row: row['País (pt-BR)'] if pd.isna(row['Capital']) else row['Capital'], axis=1
)

In [69]:
# exibindo os dados faltantes
novo_df.head()

,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital
0,AD,AND,Andorra,Andorra la Vella
1,AE,ARE,Emirados Árabes Unidos,Abu Dhabi
2,AF,AFG,Afeganistão,Kabul
3,AG,ATG,Antígua e Barbuda,St. John's
4,AI,AIA,Anguila,The Valley


In [70]:
novo_df[novo_df['Capital'].isnull()]

,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital


In [71]:
# Corrigindo o valor NaN na coluna 'ISO-3166 alpha2' para o país 'Namíbia'
novo_df.loc[novo_df['País (pt-BR)'] == 'Namíbia', 'ISO-3166 alpha2'] = 'NA'

In [72]:
print(novo_df[novo_df['País (pt-BR)'] == 'Namíbia'])

    ISO-3166 alpha2 ISO-3166 alpha3 País (pt-BR)   Capital
159              NA             NAM      Namíbia  Windhoek


In [73]:
novo_df[novo_df['ISO-3166 alpha2'].isnull()]

,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital


In [74]:
novo_df.sample(5)

,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital
186,QA,QAT,Catar,Doha
82,GI,GIB,Gibraltar,Gibraltar
232,US,USA,Estados Unidos,Washington
154,MV,MDV,Maldivas,Male
146,MN,MNG,Mongólia,Ulaanbaatar


In [75]:
# Função para gerar a URL da bandeira
def gerar_url_bandeira(alpha2):
    base_url = 'http://www.geonames.org/flags/x/'
    return f"{base_url}{alpha2.lower()}.gif"

# Adicionando a coluna de URLs das bandeiras
novo_df['URL Bandeira'] = novo_df['ISO-3166 alpha2'].apply(gerar_url_bandeira)

# Exibindo o DataFrame atualizado com as URLs das bandeiras
display(novo_df.head())

,ISO-3166 alpha2,ISO-3166 alpha3,País (pt-BR),Capital,URL Bandeira
0,AD,AND,Andorra,Andorra la Vella,http://www.geonames.org/flags/x/ad.gif
1,AE,ARE,Emirados Árabes Unidos,Abu Dhabi,http://www.geonames.org/flags/x/ae.gif
2,AF,AFG,Afeganistão,Kabul,http://www.geonames.org/flags/x/af.gif
3,AG,ATG,Antígua e Barbuda,St. John's,http://www.geonames.org/flags/x/ag.gif
4,AI,AIA,Anguila,The Valley,http://www.geonames.org/flags/x/ai.gif


In [76]:
# verificando dados faltantes
novo_df.isnull().sum()

ISO-3166 alpha2    0
ISO-3166 alpha3    0
País (pt-BR)       0
Capital            0
URL Bandeira       0
dtype: int64

In [77]:
# salvando o DataFrame em um arquivo do Excel
novo_df.to_excel('GeoDados_flags.xlsx', index=False)